In [14]:
import pandas as pd
import numpy as np

Required Datasets
RNA_DE_genes - filtered for protein_coding genes only:
`/data/reddylab/Revathy/collabs/Keith/Tcell/data/rna_seq/deseq/all/individual/Th1.Sorted.72hr_vs_mmTh0.Sorted.72hr.padj0.05.with_genes.txt`<br>

RNA_DE_gene_with_annotation:
`/data/reddylab/Revathy/collabs/Keith/Th1/data/rna_seq/counts/Th1.Sorted.DE_genes_annotated.bed`<br>

ATAC_DA_hits - with closest gene annotated:
`/data/reddylab/Revathy/collabs/Keith/Th1/data/atac_seq/deseq/Th1.96hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.padj0.05.with_closest_gene.bed`<br>

all_ATAC_peaks:
`/data/reddylab/Revathy/collabs/Keith/Th1/data/atac_seq/featureCounts/Th0_Th1.72hrs.UnionPeakSet.combined.bed`<br>

ASTARR_hits:
`/data/reddylab/Revathy/collabs/Keith/Th1/data/starr_seq/220407_Th1vsallPooledASTARR_allPooled_deseq2_q0.01.bed`

In [51]:
!wc -l /data/reddylab/Revathy/collabs/Keith/Th1/data/starr_seq/220407_Th1vsallPooledASTARR_allPooled_deseq2_q0.01.bed

2740 /data/reddylab/Revathy/collabs/Keith/Th1/data/starr_seq/220407_Th1vsallPooledASTARR_allPooled_deseq2_q0.01.bed


In [ ]:
Th1_ATAC_unionset_featurecounts = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Th1/data/atac_seq/featureCounts/Th0_Th1.72hrs.UnionPeakSet.featurecounts.txt', sep='\t')
Th1_ATAC_unionset_featurecounts_subset = Th1_ATAC_unionset_featurecounts[['#Geneid']]
Th1_ATAC_unionset_featurecounts_subset = Th1_ATAC_unionset_featurecounts_subset.rename(columns={'#Geneid':'chr_start_end'})
Th1_ATAC_unionset_featurecounts_subset[['chr','start','end']] = Th1_ATAC_unionset_featurecounts_subset['chr_start_end'].str.split('_', expand=True)
Th1_ATAC_unionset_featurecounts_bed = Th1_ATAC_unionset_featurecounts_subset[['chr','start','end','chr_start_end']]
Th1_ATAC_unionset_featurecounts_bed.to_csv('/data/reddylab/Revathy/collabs/Keith/Th1/data/atac_seq/featureCounts/Th0_Th1.72hrs.UnionPeakSet.combined.bed', sep='\t', header=None, index=False)
Th1_ATAC_unionset_featurecounts_bed

In [22]:
RNA_DE_genes = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Tcell/data/rna_seq/deseq/all/individual/Th1.Sorted.72hr_vs_mmTh0.Sorted.72hr.padj0.05.with_genes.txt', sep='\t')
RNA_DE_genes = RNA_DE_genes.rename(columns={'Unnamed: 0':'ENSID'})
TSS = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/data/annotation_files/gencode.vM19.basic.annotation.TSS.bed', sep='\t', names=['chr','start','end','ENSID','score','strand','GeneName','GeneType'])
DE_genes_annotated = RNA_DE_genes.merge(TSS, on=['ENSID','GeneName'], how='inner')
DE_genes_annotated[['chr','start','end','ENSID','log2FoldChange','padj','GeneName']].to_csv('/data/reddylab/Revathy/collabs/Keith/Th1/data/rna_seq/counts/Th1.Sorted.DE_genes_annotated.bed', sep='\t', index=False, header=None)


In [6]:
%%bash
bedtools intersect \
-a /data/reddylab/Revathy/collabs/Keith/Th1/data/atac_seq/featureCounts/Th0_Th1.72hrs.UnionPeakSet.combined.bed \
-b /data/reddylab/Revathy/collabs/Keith/Th1/data/starr_seq/220407_Th1vsallPooledASTARR_allPooled_deseq2_q0.01.bed \
-wa \
-wb \
> /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_all_ATAC_peaks_ASTARR_intersected.txt

***** WARNING: File /data/reddylab/Revathy/collabs/Keith/Th1/data/starr_seq/220407_Th1vsallPooledASTARR_allPooled_deseq2_q0.01.bed has inconsistent naming convention for record:
JH584304.1	39920	40649	1.14162854548613	4.30137917484541e-27	JH584304.1_39920_40649

***** WARNING: File /data/reddylab/Revathy/collabs/Keith/Th1/data/starr_seq/220407_Th1vsallPooledASTARR_allPooled_deseq2_q0.01.bed has inconsistent naming convention for record:
JH584304.1	39920	40649	1.14162854548613	4.30137917484541e-27	JH584304.1_39920_40649



In [7]:
!head /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_all_ATAC_peaks_ASTARR_intersected.txt

chr1	9541368	9541560	chr1_9541368_9541560	chr1	9541368	9541560	0.426969214083469	0.00591675002454843	chr1_9541368_9541560
chr1	9544942	9546044	chr1_9544942_9546044	chr1	9544913	9546078	0.432531992545172	8.83432739023274e-05	chr1_9544913_9546078
chr1	9797583	9799082	chr1_9797583_9799082	chr1	9797500	9799313	-0.721920410550132	2.89445839161276e-07	chr1_9797500_9799313
chr1	12864042	12865731	chr1_12864042_12865731	chr1	12864042	12865758	0.598991531571659	3.9511046760769e-06	chr1_12864042_12865758
chr1	23382545	23383844	chr1_23382545_23383844	chr1	23382501	23383844	-0.5066459910781	1.32939871782717e-06	chr1_23382501_23383844
chr1	24612515	24613579	chr1_24612515_24613579	chr1	24612411	24613664	2.90871622425359	2.09885525982457e-184	chr1_24612411_24613664
chr1	33882880	33883947	chr1_33882880_33883947	chr1	33882841	33884041	0.39356117684345	0.0010409731748668	chr1_33882841_33884041
chr1	33906418	33908698	chr1_33906418_33908698	chr1	33906418	33909689	-0.339902024196524	0.0044310300271126	chr1_

In [8]:
%%bash
cat /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_all_ATAC_peaks_ASTARR_intersected.txt \
| awk -vOFS='\t' '{print $1,$2<$6?$2:$6,$3<$7?$3:$7,$0}' > /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_all_ATAC_peaks_ASTARR_intersected_corrected.txt


In [25]:
ATAC_peaks_ASTARR_intersected_corrected = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_all_ATAC_peaks_ASTARR_intersected_corrected.txt', sep='\t',
                                                      names=['chr','start','end','ATAC_chr','ATAC_start','ATAC_end','ATAC_chr_start_end','ASTARR_chr','ASTARR_start','ASTARR_end','ASTARR_log2FC','ASTARR_padj','ASTARR_chr_start_end'])

ATAC_peaks_ASTARR_intersected_corrected['chr_start_end'] = ATAC_peaks_ASTARR_intersected_corrected['chr']+'_'+ATAC_peaks_ASTARR_intersected_corrected['start'].astype('str')+'_'+ATAC_peaks_ASTARR_intersected_corrected['end'].astype('str')
ATAC_peaks_ASTARR_intersected_corrected = ATAC_peaks_ASTARR_intersected_corrected[['ATAC_chr','ATAC_start','ATAC_end','ATAC_chr_start_end','ASTARR_log2FC','ASTARR_padj','ASTARR_chr_start_end','chr_start_end']]
ATAC_peaks_ASTARR_intersected_corrected.head()



,ATAC_chr,ATAC_start,ATAC_end,ATAC_chr_start_end,ASTARR_log2FC,ASTARR_padj,ASTARR_chr_start_end,chr_start_end
0,chr1,9541368,9541560,chr1_9541368_9541560,0.426969,5.916750e-03,chr1_9541368_9541560,chr1_9541368_9541560
1,chr1,9544942,9546044,chr1_9544942_9546044,0.432532,8.834327e-05,chr1_9544913_9546078,chr1_9544913_9546044
2,chr1,9797583,9799082,chr1_9797583_9799082,-0.721920,2.894458e-07,chr1_9797500_9799313,chr1_9797500_9799082
3,chr1,12864042,12865731,chr1_12864042_12865731,0.598992,3.951105e-06,chr1_12864042_12865758,chr1_12864042_12865731
4,chr1,23382545,23383844,chr1_23382545_23383844,-0.506646,1.329399e-06,chr1_23382501_23383844,chr1_23382501_23383844


ATAC DA peaks with DE genes in the 100kb region



In [23]:
%%bash
bedtools window -a /data/reddylab/Revathy/collabs/Keith/Th1/data/atac_seq/featureCounts/Th0_Th1.72hrs.UnionPeakSet.combined.bed \
-b /data/reddylab/Revathy/collabs/Keith/Th1/data/rna_seq/counts/Th1.Sorted.DE_genes_annotated.bed \
-w 100000 > /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_Th0_all_ATAC_100kb_DE_genes.txt

In [24]:
!wc -l /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_Th0_all_ATAC_100kb_DE_genes.txt

131160 /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_Th0_all_ATAC_100kb_DE_genes.txt


In [25]:
!head /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_Th0_all_ATAC_100kb_DE_genes.txt

chr1	4688892	4689633	chr1_4688892_4689633	chr1	4785738	4785739	ENSMUSG00000033845.13	0.7459060183764531	1.48159941252167e-22	Mrpl15
chr1	4724465	4725198	chr1_4724465_4725198	chr1	4785738	4785739	ENSMUSG00000033845.13	0.7459060183764531	1.48159941252167e-22	Mrpl15
chr1	4745580	4745966	chr1_4745580_4745966	chr1	4785738	4785739	ENSMUSG00000033845.13	0.7459060183764531	1.48159941252167e-22	Mrpl15
chr1	4747901	4748766	chr1_4747901_4748766	chr1	4785738	4785739	ENSMUSG00000033845.13	0.7459060183764531	1.48159941252167e-22	Mrpl15
chr1	4768368	4768600	chr1_4768368_4768600	chr1	4785738	4785739	ENSMUSG00000033845.13	0.7459060183764531	1.48159941252167e-22	Mrpl15
chr1	4768368	4768600	chr1_4768368_4768600	chr1	4857812	4857813	ENSMUSG00000033813.15	0.7841810464297521	1.6230845539169199e-19	Tcea1
chr1	4774905	4775978	chr1_4774905_4775978	chr1	4785738	4785739	ENSMUSG00000033845.13	0.7459060183764531	1.48159941252167e-22	Mrpl15
chr1	4774905	4775978	chr1_4774905_4775978	chr1	4857812	4857813	ENSMUSG00000

#### calculate the distance between the ATAC peaks and DE genes

In [8]:
%%bash
sort -k1,1 -k2,2n /data/reddylab/Revathy/collabs/Keith/Th1/data/rna_seq/counts/Th1.Sorted.DE_genes_annotated.bed \
> /data/reddylab/Revathy/collabs/Keith/Th1/data/rna_seq/counts/Th1.Sorted.DE_genes_annotated.sorted.bed
bedtools closest \
-a /data/reddylab/Revathy/collabs/Keith/Th1/data/atac_seq/featureCounts/Th0_Th1.72hrs.UnionPeakSet.combined.bed \
-b /data/reddylab/Revathy/collabs/Keith/Th1/data/rna_seq/counts/Th1.Sorted.DE_genes_annotated.sorted.bed \
-D a \
> /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_all_ATAC_peaks_distance_to_100kb_DE_genes.txt

***** WARNING: File /data/reddylab/Revathy/collabs/Keith/Th1/data/atac_seq/featureCounts/Th0_Th1.72hrs.UnionPeakSet.combined.bed has inconsistent naming convention for record:
GL456216.1	15713	17710	GL456216.1_15713_17710

***** WARNING: File /data/reddylab/Revathy/collabs/Keith/Th1/data/atac_seq/featureCounts/Th0_Th1.72hrs.UnionPeakSet.combined.bed has inconsistent naming convention for record:
GL456216.1	15713	17710	GL456216.1_15713_17710



In [12]:
!wc -l /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_all_ATAC_peaks_distance_to_100kb_DE_genes.txt

117701 /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_all_ATAC_peaks_distance_to_100kb_DE_genes.txt


In [11]:
!wc -l /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_Th0_all_ATAC_100kb_DE_genes.txt

131160 /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_Th0_all_ATAC_100kb_DE_genes.txt


In [54]:
ATAC_peaks_100kb_DE_gene_intersected = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_Th0_all_ATAC_100kb_DE_genes.txt', sep='\t',
                                                   names=['ATAC_chr','ATAC_start','ATAC_end','ATAC_chr_start_end','gene_chr','gene_start','gene_end','ENS_ID','RNA_log2FC','RNA_padj','GeneName'])

ATAC_peaks_100kb_DE_genes_distance = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_all_ATAC_peaks_distance_to_100kb_DE_genes.txt', sep='\t', 
                                                 names=['ATAC_chr','ATAC_start','ATAC_end','ATAC_chr_start_end','gene_chr','gene_start','gene_end','ENS_ID','RNA_log2FC','RNA_padj','GeneName','distance_DE_gene'])

ATAC_peaks_100kb_DE_gene_intersected_distance = ATAC_peaks_100kb_DE_gene_intersected.merge(ATAC_peaks_100kb_DE_genes_distance, on=['ATAC_chr_start_end','ENS_ID','GeneName'], how='left')

ATAC_peaks_100kb_DE_gene_intersected_distance = ATAC_peaks_100kb_DE_gene_intersected_distance[['ATAC_chr_start_end','GeneName','RNA_log2FC_x','RNA_padj_x','distance_DE_gene']]
ATAC_peaks_100kb_DE_gene_intersected_distance = ATAC_peaks_100kb_DE_gene_intersected_distance.rename(columns={'RNA_log2FC_x':'RNA_log2FC', 'RNA_padj_x':'RNA_padj'})
ATAC_peaks_100kb_DE_gene_intersected_distance.drop_duplicates().to_csv('/data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_ATAC_peaks_100kb_DE_gene_intersected_subsetted.bed', sep='\t', header=None, index=False)

len(ATAC_peaks_100kb_DE_gene_intersected_distance.ATAC_chr_start_end.unique())

/data/reddylab/software/miniconda3/envs/revathy_py3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


65498

There are 65498 unique ATAC peaks with DE genes in the 100kb window

In [31]:
%%bash
cat /data/reddylab/Revathy/collabs/Keith/Th1/data/atac_seq/featureCounts/Th0_Th1.72hrs.UnionPeakSet.combined.bed | cut -f4 \
> /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_ATAC_peaks_coordinates.txt

In [34]:
!head /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_ATAC_peaks_coordinates.txt | uniq

chr1_3670944_3671848
chr1_4491772_4492903
chr1_4496199_4497230
chr1_4571571_4572139
chr1_4600496_4601112
chr1_4671216_4672098
chr1_4688892_4689633
chr1_4724465_4725198
chr1_4745580_4745966
chr1_4747901_4748766


In [35]:
%%bash
cd /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/
sbatch -p all \
    --cpus-per-task 1 \
    --mem 8G \
    -o /data/reddylab/Revathy/collabs/Keith/Th1/logs/Th1_closest_gene_annotation_to_ATAC_peaks.out \
    <<'EOF'
#!/bin/bash
SAMPLE="Th1_ATAC_peaks_coordinates.txt"

tail -n+2 ${SAMPLE} \
| cut -f1 \
| awk -F_ -vOFS="\t" '{print $1, $2, $3, $1"_"$2"_"$3}' \
| sort -k1,1 -k2,2n \
| bedtools closest \
    -nonamecheck \
    -D a \
    -a - \
    -b /data/reddylab/Revathy/collabs/Keith/data/annotation_files/gencode.vM19.basic.annotation.TSS.bed \
| awk -vOFS="\t" \
    '{if(last==$4){genes=$11","genes;}else{if(genes){print peak, genes, distance}peak=$4; genes=$11; distance=$13; last=peak}}END{if(last==$4){genes=$11","genes;print peak, genes, distance}}' \
> ${SAMPLE/.txt/.closest_gene.txt}

EOF

Submitted batch job 28159904


In [36]:
!wc -l /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_ATAC_peaks_coordinates.closest_gene.txt

117426 /data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_ATAC_peaks_coordinates.closest_gene.txt


In [20]:
ATAC_peaks_closest_gene_intersected = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_ATAC_peaks_coordinates.closest_gene.txt', sep='\t',
                                                  names=['ATAC_chr_start_end','closest_GeneName','distance_closest_gene'])
ATAC_peaks_closest_gene_intersected = ATAC_peaks_closest_gene_intersected[['ATAC_chr_start_end','closest_GeneName','distance_closest_gene']]
ATAC_peaks_closest_gene_intersected

,ATAC_chr_start_end,closest_GeneName,distance_closest_gene
0,chr1_4491772_4492903,Sox17,4451
1,chr1_4496199_4497230,Sox17,124
2,chr1_4571571_4572139,Sox17,-74218
3,chr1_4600496_4601112,Sox17,-103143
4,chr1_4671216_4672098,Mrpl15,113641
5,chr1_4688892_4689633,Mrpl15,96106
6,chr1_4724465_4725198,Mrpl15,60541
7,chr1_4745580_4745966,Mrpl15,39773
8,chr1_4747901_4748766,Mrpl15,36973
9,chr1_4768368_4768600,Mrpl15,17139


In [21]:
ATAC_DA_peaks = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Th1/data/atac_seq/deseq/Th1.96hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.padj0.05.with_closest_gene.bed', sep='\t',
                            names=['chr','start','end','ATAC_log2FC','ATAC_padj','GeneName','distance_closest_gene'])
ATAC_DA_peaks['chr_start_end'] = ATAC_DA_peaks['chr']+'_'+ATAC_DA_peaks['start'].astype(str)+'_'+ATAC_DA_peaks['end'].astype(str)
ATAC_DA_peaks = ATAC_DA_peaks[['chr','start','end','ATAC_log2FC','ATAC_padj','chr_start_end']]
ATAC_DA_peaks.head()

,chr,start,end,ATAC_log2FC,ATAC_padj,chr_start_end
0,GL456216.1,15713,17710,0.208898,2.201750e-04,GL456216.1_15713_17710
1,GL456216.1,30989,33679,1.335651,8.101554e-12,GL456216.1_30989_33679
2,GL456216.1,38195,39383,0.590415,7.595325e-03,GL456216.1_38195_39383
3,GL456233.1,163840,164893,2.363168,2.051560e-10,GL456233.1_163840_164893
4,GL456366.1,28368,28718,-1.098065,3.738235e-03,GL456366.1_28368_28718


In [22]:
!head /data/reddylab/Revathy/collabs/Keith/Th1/data/atac_seq/featureCounts/Th0_Th1.72hrs.UnionPeakSet.combined.bed

chr1	3670944	3671848	chr1_3670944_3671848
chr1	4491772	4492903	chr1_4491772_4492903
chr1	4496199	4497230	chr1_4496199_4497230
chr1	4571571	4572139	chr1_4571571_4572139
chr1	4600496	4601112	chr1_4600496_4601112
chr1	4671216	4672098	chr1_4671216_4672098
chr1	4688892	4689633	chr1_4688892_4689633
chr1	4724465	4725198	chr1_4724465_4725198
chr1	4745580	4745966	chr1_4745580_4745966
chr1	4747901	4748766	chr1_4747901_4748766


In [27]:
## ATAC peaks

ATAC_peaks = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Th1/data/atac_seq/featureCounts/Th0_Th1.72hrs.UnionPeakSet.combined.bed', sep='\t',
                         names=['chr','start','end','chr_start_end'])


## ATAC peaks intersected with DA ATAC peaks to get logFC values of DA peaks

ATAC_peaks_with_logFC = ATAC_peaks.merge(ATAC_DA_peaks, on='chr_start_end', how='left')

ATAC_peaks_with_logFC = ATAC_peaks_with_logFC.rename(columns={'chr_x':'ATAC_chr',
                                                              'start_x':'ATAC_start',
                                                              'end_x':'ATAC_end',
                                                              'chr_start_end':'ATAC_chr_start_end'})
ATAC_peaks_with_logFC = ATAC_peaks_with_logFC[['ATAC_chr','ATAC_start','ATAC_end','ATAC_chr_start_end','ATAC_log2FC','ATAC_padj']]

## ATAC peaks with logFC intersected with STARR hits

ATAC_peaks_with_logFC_STARR_intersected = ATAC_peaks_with_logFC.merge(ATAC_peaks_ASTARR_intersected_corrected, on='ATAC_chr_start_end', how='left')
ATAC_peaks_with_logFC_STARR_intersected = ATAC_peaks_with_logFC_STARR_intersected.rename(columns={'ATAC_chr_x':'ATAC_chr','ATAC_start_x':'ATAC_start','ATAC_end_x':'ATAC_end',
                                                                                                   'ATAC_chr_y':'ATAC_ASTARR_chr','ATAC_start_y':'ATAC_ASTARR_start','ATAC_end_y':'ATAC_ASTARR_end'})
                                                                                                                                                                                                    
ATAC_peaks_with_logFC_STARR_intersected = ATAC_peaks_with_logFC_STARR_intersected[['ATAC_chr','ATAC_start','ATAC_end','ATAC_log2FC','ATAC_padj','ATAC_chr_start_end','ASTARR_log2FC','ASTARR_padj','ASTARR_chr_start_end']]

## ATAC with logFC and STARR intersected with DE genes in the 100kb region of the ATAC peak

ATAC_peaks_with_logFC_STARR_intersected_DE_gene = ATAC_peaks_with_logFC_STARR_intersected.merge(ATAC_peaks_100kb_DE_gene_intersected_distance, on='ATAC_chr_start_end', how='left')

## ATAC with logFC and STARR intersected with the closest gene

ATAC_peaks_with_logFC_STARR_intersected_DE_gene_closest_gene = ATAC_peaks_with_logFC_STARR_intersected_DE_gene.merge(ATAC_peaks_closest_gene_intersected, on='ATAC_chr_start_end', how='left')
ATAC_peaks_with_logFC_STARR_intersected_DE_gene_closest_gene.to_csv('/data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_ATAC_ASTARR_RNA_masterTable.txt', sep='\t', index=False)
ATAC_peaks_with_logFC_STARR_intersected_DE_gene_closest_gene


# masterTable.to_csv('/data/reddylab/Revathy/collabs/Keith/data/integrative/Th17/Th17_ATAC_ASTARR_RNA_masterTable_v4.bed', sep='\t', header=None, index=False)
# masterTable.to_csv('/data/reddylab/Revathy/collabs/Keith/data/integrative/Th17/Th17_ATAC_ASTARR_RNA_masterTable_v4.txt', sep='\t', index=False)


,ATAC_chr,ATAC_start,ATAC_end,ATAC_log2FC,ATAC_padj,ATAC_chr_start_end,ASTARR_log2FC,ASTARR_padj,ASTARR_chr_start_end,GeneName,RNA_log2FC,RNA_padj,distance_DE_gene,closest_GeneName,distance_closest_gene
0,chr1,3670944,3671848,NaN,NaN,chr1_3670944_3671848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,4491772,4492903,-1.554330,1.619256e-04,chr1_4491772_4492903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sox17,4451.0
2,chr1,4496199,4497230,NaN,NaN,chr1_4496199_4497230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sox17,124.0
3,chr1,4571571,4572139,NaN,NaN,chr1_4571571_4572139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sox17,-74218.0
4,chr1,4600496,4601112,1.443884,2.632457e-07,chr1_4600496_4601112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sox17,-103143.0
5,chr1,4671216,4672098,-0.815076,2.260663e-02,chr1_4671216_4672098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mrpl15,113641.0
6,chr1,4688892,4689633,NaN,NaN,chr1_4688892_4689633,NaN,NaN,NaN,Mrpl15,0.745906,1.481599e-22,96106.0,Mrpl15,96106.0
7,chr1,4724465,4725198,2.035391,1.474266e-08,chr1_4724465_4725198,NaN,NaN,NaN,Mrpl15,0.745906,1.481599e-22,60541.0,Mrpl15,60541.0
8,chr1,4745580,4745966,NaN,NaN,chr1_4745580_4745966,NaN,NaN,NaN,Mrpl15,0.745906,1.481599e-22,39773.0,Mrpl15,39773.0
9,chr1,4747901,4748766,1.076322,1.817531e-04,chr1_4747901_4748766,NaN,NaN,NaN,Mrpl15,0.745906,1.481599e-22,36973.0,Mrpl15,36973.0


# Th2

Required Datasets
RNA_DE_genes - filtered for protein_coding genes only:
`/data/reddylab/Revathy/collabs/Keith/Tcell/data/rna_seq/deseq/all/individual/Th2.Sorted.72hr_vs_mmTh0.Sorted.72hr.with_genes.txt`<br>

RNA_DE_gene_with_annotation:
`/data/reddylab/Revathy/collabs/Keith/Th2/data/rna_seq/counts/Th2.Sorted.DE_genes_annotated.bed`<br>

ATAC_DA_hits - with closest gene annotated:
`/data/reddylab/Revathy/collabs/Keith/Th2/data/atac_seq/deseq/Th2.96hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.with_closest_gene.txt`<br>

all_ATAC_peaks:
`/data/reddylab/Revathy/collabs/Keith/Th2/data/atac_seq/featureCounts/Th0_Th2.72hrs.UnionPeakSet.combined.bed`<br>

ASTARR_hits:
`/data/reddylab/Revathy/collabs/Keith/Th2/data/starr_seq/220407_Th2vsallPooledASTARR_allPooled_deseq2_q0.01.bed`

In [57]:
Th2_ATAC_unionset_featurecounts = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Th2/data/atac_seq/featureCounts/Th0_Th2.72hrs.UnionPeakSet.featurecounts.txt', sep='\t')
Th2_ATAC_unionset_featurecounts_subset = Th2_ATAC_unionset_featurecounts[['#Geneid']]
Th2_ATAC_unionset_featurecounts_subset = Th2_ATAC_unionset_featurecounts_subset.rename(columns={'#Geneid':'chr_start_end'})
Th2_ATAC_unionset_featurecounts_subset[['chr','start','end']] = Th2_ATAC_unionset_featurecounts_subset['chr_start_end'].str.split('_', expand=True)
Th2_ATAC_unionset_featurecounts_bed = Th1_ATAC_unionset_featurecounts_subset[['chr','start','end','chr_start_end']]
Th2_ATAC_unionset_featurecounts_bed.to_csv('/data/reddylab/Revathy/collabs/Keith/Th2/data/atac_seq/featureCounts/Th0_Th2.72hrs.UnionPeakSet.combined.bed', sep='\t', header=None, index=False)


In [56]:
RNA_DE_genes = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Tcell/data/rna_seq/deseq/all/individual/Th2.Sorted.72hr_vs_mmTh0.Sorted.72hr.with_genes.txt', sep='\t')
RNA_DE_genes = RNA_DE_genes.rename(columns={'Unnamed: 0':'ENSID'})
TSS = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/data/annotation_files/gencode.vM19.basic.annotation.TSS.bed', sep='\t', names=['chr','start','end','ENSID','score','strand','GeneName','GeneType'])
DE_genes_annotated = RNA_DE_genes.merge(TSS, on=['ENSID','GeneName'], how='inner')
DE_genes_annotated[['chr','start','end','ENSID','log2FoldChange','padj','GeneName']].to_csv('/data/reddylab/Revathy/collabs/Keith/Th2/data/rna_seq/counts/Th2.Sorted.DE_genes_annotated.bed', sep='\t', index=False, header=None)


In [58]:
%%bash
bedtools intersect \
-a /data/reddylab/Revathy/collabs/Keith/Th2/data/atac_seq/featureCounts/Th0_Th2.72hrs.UnionPeakSet.combined.bed \
-b /data/reddylab/Revathy/collabs/Keith/Th2/data/starr_seq/220407_Th2vsallPooledASTARR_allPooled_deseq2_q0.01.bed \
-wa \
-wb \
> /data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_all_ATAC_peaks_ASTARR_intersected.txt

cat /data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_all_ATAC_peaks_ASTARR_intersected.txt \
| awk -vOFS='\t' '{print $1,$2<$6?$2:$6,$3<$7?$3:$7,$0}' > /data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_all_ATAC_peaks_ASTARR_intersected_corrected.txt


***** WARNING: File /data/reddylab/Revathy/collabs/Keith/Th2/data/starr_seq/220407_Th2vsallPooledASTARR_allPooled_deseq2_q0.01.bed has inconsistent naming convention for record:
JH584304.1	24323	24816	1.2880187953916	3.56904379325554e-19	JH584304.1_24323_24816

***** WARNING: File /data/reddylab/Revathy/collabs/Keith/Th2/data/starr_seq/220407_Th2vsallPooledASTARR_allPooled_deseq2_q0.01.bed has inconsistent naming convention for record:
JH584304.1	24323	24816	1.2880187953916	3.56904379325554e-19	JH584304.1_24323_24816



In [81]:
!wc -l /data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_all_ATAC_peaks_ASTARR_intersected_corrected.txt

9494 /data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_all_ATAC_peaks_ASTARR_intersected_corrected.txt


In [36]:
ATAC_peaks_ASTARR_intersected_corrected = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_all_ATAC_peaks_ASTARR_intersected_corrected.txt', sep='\t',
                                                      names=['chr','start','end','ATAC_chr','ATAC_start','ATAC_end','ATAC_chr_start_end','ASTARR_chr','ASTARR_start','ASTARR_end','ASTARR_log2FC','ASTARR_padj','ASTARR_chr_start_end'])

ATAC_peaks_ASTARR_intersected_corrected['chr_start_end'] = ATAC_peaks_ASTARR_intersected_corrected['chr']+'_'+ATAC_peaks_ASTARR_intersected_corrected['start'].astype('str')+'_'+ATAC_peaks_ASTARR_intersected_corrected['end'].astype('str')
ATAC_peaks_ASTARR_intersected_corrected = ATAC_peaks_ASTARR_intersected_corrected[['ATAC_chr','ATAC_start','ATAC_end','ATAC_chr_start_end','ASTARR_log2FC','ASTARR_padj','ASTARR_chr_start_end','chr_start_end']]
ATAC_peaks_ASTARR_intersected_corrected.head()



,ATAC_chr,ATAC_start,ATAC_end,ATAC_chr_start_end,ASTARR_log2FC,ASTARR_padj,ASTARR_chr_start_end,chr_start_end
0,chr1,4785129,4786415,chr1_4785129_4786415,-0.470340,0.007256,chr1_4785097_4786415,chr1_4785097_4786415
1,chr1,4807119,4808718,chr1_4807119_4808718,-0.732116,0.000051,chr1_4806911_4808718,chr1_4806911_4808718
2,chr1,4857089,4858727,chr1_4857089_4858727,-0.403075,0.000114,chr1_4857038_4858897,chr1_4857038_4858727
3,chr1,5082611,5083716,chr1_5082611_5083716,-0.518030,0.000524,chr1_5082587_5083716,chr1_5082587_5083716
4,chr1,6214124,6215613,chr1_6214124_6215613,-0.579428,0.000488,chr1_6214033_6215613,chr1_6214033_6215613


ATAC DA peaks with DE genes in the 100kb region

In [61]:
%%bash
bedtools window -a /data/reddylab/Revathy/collabs/Keith/Th2/data/atac_seq/featureCounts/Th0_Th2.72hrs.UnionPeakSet.combined.bed \
-b /data/reddylab/Revathy/collabs/Keith/Th2/data/rna_seq/counts/Th2.Sorted.DE_genes_annotated.bed \
-w 100000 > /data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_Th0_all_ATAC_100kb_DE_genes.txt

In [62]:
!wc -l /data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_Th0_all_ATAC_100kb_DE_genes.txt
!head /data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_Th0_all_ATAC_100kb_DE_genes.txt

187538 /data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_Th0_all_ATAC_100kb_DE_genes.txt
chr1	4688892	4689633	chr1_4688892_4689633	chr1	4785738	4785739	ENSMUSG00000033845.13	0.421906691293139	3.33224664411683e-13	Mrpl15
chr1	4724465	4725198	chr1_4724465_4725198	chr1	4785738	4785739	ENSMUSG00000033845.13	0.421906691293139	3.33224664411683e-13	Mrpl15
chr1	4724465	4725198	chr1_4724465_4725198	chr1	4807821	4807822	ENSMUSG00000025903.14	0.0829154796856068	0.57677131964491	Lypla1
chr1	4745580	4745966	chr1_4745580_4745966	chr1	4785738	4785739	ENSMUSG00000033845.13	0.421906691293139	3.33224664411683e-13	Mrpl15
chr1	4745580	4745966	chr1_4745580_4745966	chr1	4807821	4807822	ENSMUSG00000025903.14	0.0829154796856068	0.57677131964491	Lypla1
chr1	4747901	4748766	chr1_4747901_4748766	chr1	4785738	4785739	ENSMUSG00000033845.13	0.421906691293139	3.33224664411683e-13	Mrpl15
chr1	4747901	4748766	chr1_4747901_4748766	chr1	4807821	4807822	ENSMUSG00000025903.14	0.0829154796856068	0.5767713196449

In [28]:
%%bash
sort -k1,1 -k2,2n /data/reddylab/Revathy/collabs/Keith/Th2/data/rna_seq/counts/Th2.Sorted.DE_genes_annotated.bed \
> /data/reddylab/Revathy/collabs/Keith/Th2/data/rna_seq/counts/Th2.Sorted.DE_genes_annotated.sorted.bed
bedtools closest \
-a /data/reddylab/Revathy/collabs/Keith/Th2/data/atac_seq/featureCounts/Th0_Th2.72hrs.UnionPeakSet.combined.bed \
-b /data/reddylab/Revathy/collabs/Keith/Th2/data/rna_seq/counts/Th2.Sorted.DE_genes_annotated.sorted.bed \
-D a \
> /data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_all_ATAC_peaks_distance_to_100kb_DE_genes.txt

***** WARNING: File /data/reddylab/Revathy/collabs/Keith/Th2/data/atac_seq/featureCounts/Th0_Th2.72hrs.UnionPeakSet.combined.bed has inconsistent naming convention for record:
GL456216.1	15713	17710	GL456216.1_15713_17710

***** WARNING: File /data/reddylab/Revathy/collabs/Keith/Th2/data/atac_seq/featureCounts/Th0_Th2.72hrs.UnionPeakSet.combined.bed has inconsistent naming convention for record:
GL456216.1	15713	17710	GL456216.1_15713_17710



In [31]:
!head /data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_all_ATAC_peaks_distance_to_100kb_DE_genes.txt

chr1	3670944	3671848	chr1_3670944_3671848	chr1	4785738	4785739	ENSMUSG00000033845.13	0.421906691293139	3.33224664411683e-13	Mrpl15	1113891
chr1	4491772	4492903	chr1_4491772_4492903	chr1	4785738	4785739	ENSMUSG00000033845.13	0.421906691293139	3.33224664411683e-13	Mrpl15	292836
chr1	4496199	4497230	chr1_4496199_4497230	chr1	4785738	4785739	ENSMUSG00000033845.13	0.421906691293139	3.33224664411683e-13	Mrpl15	288509
chr1	4571571	4572139	chr1_4571571_4572139	chr1	4785738	4785739	ENSMUSG00000033845.13	0.421906691293139	3.33224664411683e-13	Mrpl15	213600
chr1	4600496	4601112	chr1_4600496_4601112	chr1	4785738	4785739	ENSMUSG00000033845.13	0.421906691293139	3.33224664411683e-13	Mrpl15	184627
chr1	4671216	4672098	chr1_4671216_4672098	chr1	4785738	4785739	ENSMUSG00000033845.13	0.421906691293139	3.33224664411683e-13	Mrpl15	113641
chr1	4688892	4689633	chr1_4688892_4689633	chr1	4785738	4785739	ENSMUSG00000033845.13	0.421906691293139	3.33224664411683e-13	Mrpl15	96106
chr1	4724465	4725198	chr1_4724465_

In [34]:
ATAC_peaks_100kb_DE_gene_intersected = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_Th0_all_ATAC_100kb_DE_genes.txt', sep='\t',
                                                   names=['ATAC_chr','ATAC_start','ATAC_end','ATAC_chr_start_end','gene_chr','gene_start','gene_end','ENS_ID','RNA_log2FC','RNA_padj','GeneName'])

ATAC_peaks_100kb_DE_genes_distance = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_all_ATAC_peaks_distance_to_100kb_DE_genes.txt', sep='\t', 
                                                 names=['ATAC_chr','ATAC_start','ATAC_end','ATAC_chr_start_end','gene_chr','gene_start','gene_end','ENS_ID','RNA_log2FC','RNA_padj','GeneName','distance_DE_gene'])

ATAC_peaks_100kb_DE_gene_intersected_distance = ATAC_peaks_100kb_DE_gene_intersected.merge(ATAC_peaks_100kb_DE_genes_distance, on=['ATAC_chr_start_end','ENS_ID','GeneName'], how='left')

ATAC_peaks_100kb_DE_gene_intersected_distance = ATAC_peaks_100kb_DE_gene_intersected_distance[['ATAC_chr_start_end','GeneName','RNA_log2FC_x','RNA_padj_x','distance_DE_gene']]
ATAC_peaks_100kb_DE_gene_intersected_distance = ATAC_peaks_100kb_DE_gene_intersected_distance.rename(columns={'RNA_log2FC_x':'RNA_log2FC', 'RNA_padj_x':'RNA_padj'})

ATAC_peaks_100kb_DE_gene_intersected_distance = ATAC_peaks_100kb_DE_gene_intersected_distance[['ATAC_chr_start_end','GeneName','RNA_log2FC','RNA_padj','distance_DE_gene']]
ATAC_peaks_100kb_DE_gene_intersected_distance.drop_duplicates().to_csv('/data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_ATAC_peaks_100kb_DE_gene_intersected_subsetted.bed', sep='\t', header=None, index=False)

len(ATAC_peaks_100kb_DE_gene_intersected_distance.ATAC_chr_start_end.unique())

/data/reddylab/software/miniconda3/envs/revathy_py3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


75863

There are 75863 unique ATAC peaks with DE genes in the 100kb window

In [65]:
%%bash
cat /data/reddylab/Revathy/collabs/Keith/Th2/data/atac_seq/featureCounts/Th0_Th2.72hrs.UnionPeakSet.combined.bed | cut -f4 \
> /data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_ATAC_peaks_coordinates.txt


In [69]:
%%bash
cd /data/reddylab/Revathy/collabs/Keith/Th2/data/integration/
sbatch -p all \
    --cpus-per-task 1 \
    --mem 8G \
    -o /data/reddylab/Revathy/collabs/Keith/Th2/logs/Th2_closest_gene_annotation_to_ATAC_peaks.out \
    <<'EOF'
#!/bin/bash
SAMPLE="Th2_ATAC_peaks_coordinates.txt"

tail -n+2 ${SAMPLE} \
| cut -f1 \
| awk -F_ -vOFS="\t" '{print $1, $2, $3, $1"_"$2"_"$3}' \
| sort -k1,1 -k2,2n \
| bedtools closest \
    -nonamecheck \
    -D a \
    -a - \
    -b /data/reddylab/Revathy/collabs/Keith/data/annotation_files/gencode.vM19.basic.annotation.TSS.bed \
| awk -vOFS="\t" \
    '{if(last==$4){genes=$11","genes;}else{if(genes){print peak, genes, distance}peak=$4; genes=$11; distance=$13; last=peak}}END{if(last==$4){genes=$11","genes;print peak, genes, distance}}' \
> ${SAMPLE/.txt/.closest_gene.txt}

EOF


Submitted batch job 28159913


In [71]:
!wc -l /data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_ATAC_peaks_coordinates.closest_gene.txt

117426 /data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_ATAC_peaks_coordinates.closest_gene.txt


In [35]:
ATAC_peaks_closest_gene_intersected = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_ATAC_peaks_coordinates.closest_gene.txt', sep='\t',
                                                  names=['ATAC_chr_start_end','closest_GeneName','distance_closest_gene'])
ATAC_peaks_closest_gene_intersected = ATAC_peaks_closest_gene_intersected[['ATAC_chr_start_end','closest_GeneName','distance_closest_gene']]


ATAC_DA_peaks = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Th2/data/atac_seq/deseq/Th2.96hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.padj0.05.with_closest_gene.bed', sep='\t',
                            names=['chr','start','end','ATAC_log2FC','ATAC_padj','GeneName','distance'])
ATAC_DA_peaks['chr_start_end'] = ATAC_DA_peaks['chr']+'_'+ATAC_DA_peaks['start'].astype(str)+'_'+ATAC_DA_peaks['end'].astype(str)
ATAC_DA_peaks = ATAC_DA_peaks[['chr','start','end','ATAC_log2FC','ATAC_padj','chr_start_end']]
ATAC_DA_peaks.head()

,chr,start,end,ATAC_log2FC,ATAC_padj,chr_start_end
0,GL456216.1,13778,13946,-0.670476,2.252353e-02,GL456216.1_13778_13946
1,GL456216.1,15713,17546,-0.906952,3.412153e-11,GL456216.1_15713_17546
2,GL456233.1,159366,160130,-1.273075,8.194240e-05,GL456233.1_159366_160130
3,GL456233.1,38727,39570,-1.239184,1.628873e-06,GL456233.1_38727_39570
4,GL456239.1,33961,39941,1.320609,8.053309e-08,GL456239.1_33961_39941


In [43]:
## ATAC peaks

ATAC_peaks = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Th2/data/atac_seq/featureCounts/Th0_Th2.72hrs.UnionPeakSet.combined.bed', sep='\t',
                         names=['chr','start','end','chr_start_end'])


## ATAC peaks intersected with DA ATAC peaks to get logFC values of DA peaks

ATAC_peaks_with_logFC = ATAC_peaks.merge(ATAC_DA_peaks, on='chr_start_end', how='left')

ATAC_peaks_with_logFC = ATAC_peaks_with_logFC.rename(columns={'chr_x':'ATAC_chr',
                                                              'start_x':'ATAC_start',
                                                              'end_x':'ATAC_end',
                                                              'chr_start_end':'ATAC_chr_start_end'})
ATAC_peaks_with_logFC = ATAC_peaks_with_logFC[['ATAC_chr','ATAC_start','ATAC_end','ATAC_chr_start_end','ATAC_log2FC','ATAC_padj']]

## ATAC peaks with logFC intersected with STARR hits

ATAC_peaks_with_logFC_STARR_intersected = ATAC_peaks_with_logFC.merge(ATAC_peaks_ASTARR_intersected_corrected, on='ATAC_chr_start_end', how='left')
ATAC_peaks_with_logFC_STARR_intersected = ATAC_peaks_with_logFC_STARR_intersected.rename(columns={'ATAC_chr_x':'ATAC_chr','ATAC_start_x':'ATAC_start','ATAC_end_x':'ATAC_end',
                                                                                                   'ATAC_chr_y':'ATAC_ASTARR_chr','ATAC_start_y':'ATAC_ASTARR_start','ATAC_end_y':'ATAC_ASTARR_end'})
                                                                                                                                                                                                    
ATAC_peaks_with_logFC_STARR_intersected = ATAC_peaks_with_logFC_STARR_intersected[['ATAC_chr','ATAC_start','ATAC_end','ATAC_log2FC','ATAC_padj','ATAC_chr_start_end','ASTARR_log2FC','ASTARR_padj','ASTARR_chr_start_end']]

## ATAC with logFC and STARR intersected with DE genes in the 100kb region of the ATAC peak

ATAC_peaks_with_logFC_STARR_intersected_DE_gene = ATAC_peaks_with_logFC_STARR_intersected.merge(ATAC_peaks_100kb_DE_gene_intersected_distance, on='ATAC_chr_start_end', how='left')

## ATAC with logFC and STARR intersected with the closest gene

ATAC_peaks_with_logFC_STARR_intersected_DE_gene_closest_gene = ATAC_peaks_with_logFC_STARR_intersected_DE_gene.merge(ATAC_peaks_closest_gene_intersected, on='ATAC_chr_start_end', how='left')
ATAC_peaks_with_logFC_STARR_intersected_DE_gene_closest_gene.to_csv('/data/reddylab/Revathy/collabs/Keith/Th2/data/integration/Th2_ATAC_ASTARR_RNA_masterTable.txt', sep='\t', index=False)
ATAC_peaks_with_logFC_STARR_intersected_DE_gene_closest_gene


# masterTable.to_csv('/data/reddylab/Revathy/collabs/Keith/data/integrative/Th17/Th17_ATAC_ASTARR_RNA_masterTable_v4.bed', sep='\t', header=None, index=False)
# masterTable.to_csv('/data/reddylab/Revathy/collabs/Keith/data/integrative/Th17/Th17_ATAC_ASTARR_RNA_masterTable_v4.txt', sep='\t', index=False)


,ATAC_chr,ATAC_start,ATAC_end,ATAC_log2FC,ATAC_padj,ATAC_chr_start_end,ASTARR_log2FC,ASTARR_padj,ASTARR_chr_start_end,GeneName,RNA_log2FC,RNA_padj,distance_DE_gene,closest_GeneName,distance_closest_gene
0,chr1,3670944,3671848,NaN,NaN,chr1_3670944_3671848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,4491772,4492903,NaN,NaN,chr1_4491772_4492903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sox17,4451.0
2,chr1,4496199,4497230,NaN,NaN,chr1_4496199_4497230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sox17,124.0
3,chr1,4571571,4572139,NaN,NaN,chr1_4571571_4572139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sox17,-74218.0
4,chr1,4600496,4601112,NaN,NaN,chr1_4600496_4601112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sox17,-103143.0
5,chr1,4671216,4672098,-2.904328,1.574524e-12,chr1_4671216_4672098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mrpl15,113641.0
6,chr1,4688892,4689633,-1.660644,2.710789e-03,chr1_4688892_4689633,NaN,NaN,NaN,Mrpl15,0.421907,3.332247e-13,96106.0,Mrpl15,96106.0
7,chr1,4724465,4725198,NaN,NaN,chr1_4724465_4725198,NaN,NaN,NaN,Mrpl15,0.421907,3.332247e-13,60541.0,Mrpl15,60541.0
8,chr1,4724465,4725198,NaN,NaN,chr1_4724465_4725198,NaN,NaN,NaN,Lypla1,0.082915,5.767713e-01,NaN,Mrpl15,60541.0
9,chr1,4745580,4745966,NaN,NaN,chr1_4745580_4745966,NaN,NaN,NaN,Mrpl15,0.421907,3.332247e-13,39773.0,Mrpl15,39773.0


In [44]:
len(ATAC_peaks_with_logFC_STARR_intersected_DE_gene_closest_gene['ATAC_chr_start_end'][(ATAC_peaks_with_logFC_STARR_intersected_DE_gene_closest_gene['ATAC_log2FC'].notna())&(ATAC_peaks_with_logFC_STARR_intersected_DE_gene_closest_gene['RNA_log2FC'].notna())].unique())




9423

In [80]:
!wc -l /data/reddylab/Revathy/collabs/Keith/Th2/data/starr_seq/220407_Th2vsallPooledASTARR_allPooled_deseq2_q0.01.bed

9570 /data/reddylab/Revathy/collabs/Keith/Th2/data/starr_seq/220407_Th2vsallPooledASTARR_allPooled_deseq2_q0.01.bed


### Stats:

#### Th1: 

- Number of ATAC peaks (from the featurecounts) = 117427
- Number of DA ATAC peaks (padj.<0.05) = 43880
- Number of protein coding DE genes = 7073
- Number of ASTARRseq hits = 2740
- Number of ATAC peaks that intersect with ASTARR peaks = 997 unique DA + 1757 unique non-DA
- Number of unique ATAC peaks that have DE genes in the 100kb region = 65498
- Number of unique DA ATAC peaks with DE genes in the 100kb region = 23812


#### Th2: 

- Number of ATAC peaks (from the featurecounts) = 117427
- Number of DA ATAC peaks (padj.<0.05) = 74159
- Number of protein coding DE genes = 10791
- Number of ASTARRseq hits = 9570
- Number of ATAC peaks that intersect with ASTARR peaks = 1554 unique DA peaks + 7940 unique non-DA peaks
- Number of unique ATAC peaks that have DE genes in the 100kb region = 75863
- Number of unique DA ATAC peaks with DE genes in the 100kb region = 9423


In [49]:
import pandas as pd

th1 = pd.read_csv('/data/reddylab/Revathy/collabs/Keith/Th1/data/integration/Th1_ATAC_ASTARR_RNA_masterTable.txt', sep='\t')
len(th1['ATAC_chr_start_end'][th1['ATAC_log2FC'].notna()].unique())

43880

In [57]:
len(th1['ATAC_chr_start_end'][(th1['ATAC_log2FC'].notna())&(th1['RNA_log2FC'].notna())].unique())




23812